# Multi-Agent System - Nutrition Assessment System
This notebook demonstrates how to build a multi-agent clinical system using IntelliNode, where:

- **Agent 1**: Analyzes client notes to identify nutrition needs and health priorities.
- **Agent 2**: Creates personalized meal plan based on the assessment.

**Objectives**

In this tutorial, you will learn how to:
- Connect multiple AI providers in a single workflow by integrating OpenAI's GPT-4 and Anthropic's Claude
- Build a graph-based multi-agent system where agents pass data through defined dependencies
- Implement a practical healthcare workflow.


## Setup and Imports

In [47]:
!pip install intelli python-dotenv

In [49]:
import os
import asyncio
from dotenv import load_dotenv
from intelli.flow import Agent, Task, Flow, TextTaskInput, AgentTypes
import json

In [51]:
# Load environment variables
load_dotenv()

# Get API keys
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_KEY = os.getenv("ANTHROPIC_API_KEY")

In [53]:
print(f"OpenAI Key: {'Available' if OPENAI_KEY else 'Error to load the key'}")
print(f"Anthropic Key: {'Available' if ANTHROPIC_KEY else 'Error to load the key'}")

OpenAI Key: Available
Anthropic Key: Available


## Client Case Notes
A brief profile of the user seeking nutrition guidance.

In [56]:
case_notes = """
Client: Sarah Chen, 42-year-old female
Weight: 180 lbs, Height: 5'4
Health: Pre-diabetic, high blood pressure (controlled with medication)
Current diet: Skips breakfast, buys lunch, eats late dinners
Activity: Walks occasionally, sits most of the day, sleeps poorly
Allergies: Lactose intolerant, allergic to shellfish
Goals: Lose 20 pounds, prevent diabetes, have more energy
Preferences: Loves Asian and Mediterranean food, dislikes fish, needs quick meals
Budget: Moderate, prefers affordable ingredients
"""

## Create Agents
Set up two specialized agents - one for assessment and one for planning.

In [58]:
# Agent 1: Nutrition Assessor (OpenAI)
nutrition_assessor = Agent(
    agent_type=AgentTypes.TEXT.value,
    provider="openai",
    mission="Extract nutrition-relevant information from client notes",
    model_params={"key": OPENAI_KEY, "model": "gpt-4.1"}
)

# Agent 2: Meal Planner (Anthropic)
meal_planner = Agent(
    agent_type=AgentTypes.TEXT.value,
    provider="anthropic",
    mission="Create personalized meal plans",
    model_params={"key": ANTHROPIC_KEY, "model": "claude-sonnet-4-20250514"}
)

## Create Tasks
Define what each agent should do.

In [61]:
# Task 1: Assess nutrition needs
assessment_task = Task(
    TextTaskInput(f"""
    Review this client case and extract:
    - Health conditions to consider
    - Current eating problems
    - Allergies and restrictions
    - Personal goals
    - Food preferences
    - Main nutrition priorities

    Case details:
    {case_notes}
    """),
    nutrition_assessor,
    log=True
)

In [65]:
# Task 2: Create meal plan
planning_task = Task(
    TextTaskInput(
        "Create a simple 3-day meal plan for weight loss and diabetes prevention. "
        "Include easy breakfast, lunch, dinner, and one snack per day."
    ),
    meal_planner,
    log=True
)

## Create Flow
Connect the agents so the assessment flows into the meal planning.

In [70]:
flow = Flow(
    tasks={
        "assessment_task": assessment_task,
        "planning_task": planning_task
    },
    map_paths={
        "assessment_task": ["planning_task"],
        "planning_task": []
    },
    log=True
)

In [72]:
# generate flow graph
flow.generate_graph_img(
    name="nutrition_flow",
    save_path="./output/"
)

'./output/nutrition_flow.png'

## Run the Flow

In [74]:
async def run_nutrition_workflow():
    results = await flow.start()

    return results

In [76]:
results = await run_nutrition_workflow()

---- Execute task assessment_task (text/openai) ----
- The task output head: : **Health conditions to consider**
- Pre-diabetes
- Hypertension (high blood pressure, controlled with medication)

**Current eating problems**
- Skips breakfast regularly
- Relies on store-bought or t
Task assessment_task executed successfully
Task assessment_task output type: text
---- Execute task planning_task (text/anthropic) ----
Task text expects input type: text
Found matching input type with 1 outputs
- Inside the task with input data head: : **Health conditions to consider**
- Pre-diabetes
- Hypertension (high blood pressure, controlled with medication)

**Current eating problems**
- Skips breakfast regularly
- Relies on store-bought or t
- Input data with template: : PREVIOUS_ANALYSIS: {0}
CURRENT_TASK: Create a simple 3-day meal plan for weight loss and diabetes prevention. Include easy breakfast, lunch, dinner, and one snack per day.

**Health conditions to cons
- The task output head: : # 3-Day 

In [77]:
print("=== NUTRITION ASSESSMENT (OpenAI) ===")
print(results["assessment_task"]["output"])

=== NUTRITION ASSESSMENT (OpenAI) ===
**Health conditions to consider**
- Pre-diabetes
- Hypertension (high blood pressure, controlled with medication)

**Current eating problems**
- Skips breakfast regularly
- Relies on store-bought or takeout lunches (unknown nutritional quality)
- Eats late dinners
- Poor sleep quality
- Minimal physical activity

**Allergies and restrictions**
- Lactose intolerant
- Shellfish allergy

**Personal goals**
- Lose 20 pounds
- Prevent progression to diabetes
- Increase energy levels

**Food preferences**
- Enjoys Asian and Mediterranean cuisines
- Dislikes fish
- Needs quick, easy meal options
- Prefers affordable ingredients

**Main nutrition priorities**
- Support stable blood sugar (low-glycemic, high-fiber foods)
- Focus on heart-healthy eating (DASH or Mediterranean-influenced, low sodium, minimal processed foods)
- Manage weight through calorie balance and portion control
- Ensure lactose-free and shellfish-free meal planning
- Increase meal regul

In [78]:
print("\n\n=== MEAL PLAN (Anthropic) ===")
print(results["planning_task"]["output"])



=== MEAL PLAN (Anthropic) ===
# 3-Day Meal Plan for Weight Loss & Diabetes Prevention

*Designed for pre-diabetes, hypertension management, lactose-free, shellfish-free*

## **DAY 1**

### 🌅 **BREAKFAST** (7-8 AM)
**Mediterranean Veggie Scramble**
- 2 eggs scrambled with spinach, diced tomatoes, and bell peppers
- 1 slice whole grain toast
- 1/2 avocado, sliced
- Herbal tea or black coffee (unsweetened)

*Prep tip: Pre-chop vegetables Sunday for quick weekday cooking*

### 🥗 **LUNCH** (12-1 PM)
**Asian-Style Chicken & Vegetable Bowl**
- 4 oz grilled chicken breast (seasoned with ginger, garlic, low-sodium soy sauce)
- 1 cup steamed broccoli and snap peas
- 1/2 cup brown rice
- 1 tbsp sesame seeds
- Side: cucumber slices with rice vinegar

### 🍎 **SNACK** (3-4 PM)
- 1 medium apple with 2 tbsp almond butter

### 🍽️ **DINNER** (6-7 PM)
**Mediterranean Stuffed Bell Peppers**
- 2 bell peppers stuffed with ground turkey, quinoa, diced tomatoes, onions, and herbs
- Side salad with mixed gre